In [1]:
import tensorflow as tf
from keras.layers import Conv2D, Input, Dense, BatchNormalization, MaxPool2D, GlobalAvgPool2D, Flatten, AveragePooling2D, ReLU, Add, ZeroPadding2D
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
!unzip gdrive/MyDrive/Dataset/GTSRB/test.zip > /dev/null
!unzip gdrive/MyDrive/Dataset/GTSRB/train.zip > /dev/null
!unzip gdrive/MyDrive/Dataset/GTSRB/val.zip > /dev/null

In [4]:
def create_generator(batch_size):
    train_preprocessor = ImageDataGenerator(
        rescale=1 / 255
    )

    train_generator = train_preprocessor.flow_from_directory(
        '/content/train',
        class_mode='categorical',
        target_size=(60, 60),
        color_mode='rgb',
        shuffle=True,
        batch_size=batch_size
    )

    val_generator = train_preprocessor.flow_from_directory(
        '/content/val',
        class_mode='categorical',
        target_size=(60, 60),
        color_mode='rgb',
        shuffle=False,
        batch_size=batch_size
    )

    test_generator = train_preprocessor.flow_from_directory(
        '/content/Test',
        class_mode='categorical',
        target_size=(60, 60),
        color_mode='rgb',
        shuffle=False,
        batch_size=batch_size
    )

    return train_generator, val_generator, test_generator

In [5]:
def identity_block(x, filters):
    x_shortcut = x
    f1, f2, f3 = filters

    x = Conv2D(filters=f1, kernel_size=(1, 1), strides=(1, 1), padding='valid')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = Conv2D(filters=f2, kernel_size=(3, 3), strides=(1, 1), padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = Conv2D(filters=f3, kernel_size=(1, 1), strides=(1, 1), padding='valid')(x)
    x = BatchNormalization()(x)

    x = Add()([x, x_shortcut])
    x = ReLU()(x)

    return x


def convolutional_block(x, filters, s):
    x_shortcut = x
    f1, f2, f3 = filters

    x = Conv2D(filters=f1, kernel_size=(1, 1), strides=(s, s), padding='valid')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = Conv2D(filters=f2, kernel_size=(3, 3), strides=(1, 1), padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = Conv2D(filters=f3, kernel_size=(1, 1), strides=(1, 1), padding='valid')(x)
    x = BatchNormalization()(x)

    x_shortcut = Conv2D(filters=f3, kernel_size=(1, 1), strides=(s, s), padding='valid')(x_shortcut)
    x_shortcut = BatchNormalization()(x_shortcut)

    x = Add()([x, x_shortcut])
    x = ReLU()(x)

    return x


def resnet50(input_shape, classes):
    my_input = Input(shape=input_shape)

    x = ZeroPadding2D(padding=(3, 3))(my_input)

    x = Conv2D(64, kernel_size=(7, 7), strides=(2, 2))(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = MaxPool2D(pool_size=(3, 3), strides=(2, 2))(x)

    x = convolutional_block(x, (64, 64, 256), 1)
    x = identity_block(x, (64, 64, 256))
    x = identity_block(x, (64, 64, 256))

    x = convolutional_block(x, (128, 128, 512), 2)
    x = identity_block(x, (128, 128, 512))
    x = identity_block(x, (128, 128, 512))
    x = identity_block(x, (128, 128, 512))

    x = convolutional_block(x, (256, 256, 1024), 2)
    x = identity_block(x, (256, 256, 1024))
    x = identity_block(x, (256, 256, 1024))
    x = identity_block(x, (256, 256, 1024))
    x = identity_block(x, (256, 256, 1024))
    x = identity_block(x, (256, 256, 1024))

    x = convolutional_block(x, (512, 512, 2048), 2)
    x = identity_block(x, (512, 512, 2048))
    x = identity_block(x, (512, 512, 2048))

    x = AveragePooling2D(pool_size=(2, 2))(x)
    x = Flatten()(x)
    x = Dense(classes, activation='softmax')(x)

    model = tf.keras.Model(inputs=my_input, outputs=x)

    return model

In [ ]:
batch_size = 32
epochs = 10

train_generator, val_generator, test_generator = create_generator(batch_size)

model = resnet50(train_generator.image_shape, train_generator.num_classes)
# model.summary()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_generator, epochs=10, batch_size=32, validation_data=val_generator)